In [ ]:
#This notebook is for testing ipython widgets to make a GUI for the GREAT Exposure Mapping Program

In [ ]:
#Import necessary libraries
import sens_map_lib #main library for the sensitivity |mapper
import numpy as np #numpy
from astropy.modeling import models #For generating synthetic signal on the sky
from matplotlib import pyplot as plt #For plotting

import ipywidgets as widgets #For widgets
from ipywidgets import Layout, HBox, VBox, Label
%matplotlib notebook

from IPython.core.display import display, HTML #Make width of notebook 100% of page
display(HTML("<style>.container { width:100% !important; }</style>"))